# **End Phase 2 Project**

In [5]:
# importing necessary libraries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import statistics

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

from scipy.stats import ttest_ind, ttest_1samp, chi2_contingency, norm

from sklearn.preprocessing import StandardScaler, LabelEncoder

import os
import pickle

import warnings
warnings.filterwarnings('ignore')

## Data Loading  

We begin by loading all datasets:
- SQLite database (`im.db`) containing IMDb-like tables.  
- CSV/TSV files providing box office, TMDB, Rotten Tomatoes, and budget information.  

These diverse sources will later be merged into a unified dataset for analysis.


In [6]:
import sqlite3
conn = sqlite3.connect(r"C:\Users\PC\Desktop\Group_3_Project\im.db")
cursor = conn.cursor()

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


# Loading Relevant Tables and Data

In [28]:
# Load main tables from the database
movie_basics = pd.read_sql_query("SELECT * FROM movie_basics;", conn)
movie_ratings = pd.read_sql_query("SELECT * FROM movie_ratings;", conn)

# Load CSV and TSV files
movie_gross = pd.read_csv(r"C:\Users\PC\Documents\Phase-2-project-G3\Data\bom.movie_gross.csv")
tmdb = pd.read_csv(r"C:\Users\PC\Documents\Phase-2-project-G3\Data\tmdb.movies.csv")
movie_budgets = pd.read_csv(r"C:\Users\PC\Documents\Phase-2-project-G3\Data\tn.movie_budgets.csv")
rt_movie = pd.read_csv(r"C:\Users\PC\Documents\Phase-2-project-G3\Data\rt.movie_info.tsv", sep='\t')
rt_reviews = pd.read_csv(r"C:\Users\PC\Documents\Phase-2-project-G3\Data\rt.reviews.tsv", sep='\t', encoding='latin1')
